README:
This is the python parameterization pipeline that we used to analyze the behavior of our two recommendation methods. The beginning of each cell has a comment at the top that explains the purpose of the cell. However, a brief synopsis is provided here as well.
The only supporting code this pynotebook uses are listed in the "import" cell (directly below this cell)

1: Translate the MATLAB functions into python

2: Define two python Classes in terms of the translated functions, so that the two methods can be tested and compared using scikit-learn's GridSearchCV function

3: Load the four datasets that were pre-processed in MATLAB

4: Run GridSearchCV on the two different methods with each of the four datasets, varying the rank value for the NNMF method, and the fraction of variance explained (FoV) for the PCA by SVD method.

5: Report the resulting RMSE values for each parameter in both methods, four all four datasets

In [9]:
# Import relevant functions
import numpy as np
import pandas as pd
import random as rand
import scipy.io
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score

In [10]:
# These are the python implementations of the MATLAB functions we used for our recommendation system.
# As far as we have tested, they are the same as the MATLAB functions, though with different syntax
# for obvious reasons.
# Because these functions are just coppied over from MATLAB, they are left uncommented

def svd_deconstruction(X, defaultVals, colMeans, rowMeans):        
    if (colMeans == 0 and rowMeans == 0):
        X[X==0] = defaultVals
        
    elif (colMeans == 1 and rowMeans == 0):
       for k in range(len(defaultVals)):
           logicalIndex = X[:,k] == 0
           X[logicalIndex] = defaultVals[k]
       
    elif (colMeans == 0 and rowMeans == 1):
        for k in range(len(defaultVals)):
           logicalIndex = X[k,:] == 0
           X[logicalIndex] = defaultVals[k]
           
    userCount = X.shape[1]
    meanVec = X.mean()
    xTilde = X - meanVec
    xTildeScaled = (1/np.sqrt(userCount))*xTilde
    W, S, V = np.linalg.svd(xTildeScaled, full_matrices=False)
    pcMatrix = np.matmul(np.transpose(W), xTilde)
    lambdas = np.multiply(S, S)
    sortedLambdaVec = np.diag(lambdas)
    return pcMatrix, W, sortedLambdaVec


def KPrincipled(eigenvalueVector, principledThreshold):
    fractionOfVariance = np.zeros(len(eigenvalueVector))
    total = np.sum(eigenvalueVector)
    cummulative = np.cumsum(eigenvalueVector)
    fractionOfVariance = np.divide(cummulative, total)
    
    idx = 0
    while fractionOfVariance[idx] < principledThreshold:
        idx += 1
    
    return idx


def lowRankMatrixApproximator(muX, W, pcMatrix, k):
    predictedListensMatrix = np.transpose(muX + np.matmul(W[:,:k], pcMatrix[:k,:]))
    predictedListensMatrix[predictedListensMatrix < 0] = 0
    
    return predictedListensMatrix


def recommend(X, threshold = 0.9, defaultVals=0, colMeans=0, rowMeans=0):
    pcMatrix, W, sortedLambdaVec = svd_deconstruction(X, defaultVals, colMeans, rowMeans)
    principledK = KPrincipled(sortedLambdaVec, threshold)
    predictedListensMatrix = lowRankMatrixApproximator(X.mean(), W, pcMatrix, principledK)
    return predictedListensMatrix


def listen2Rating(X):
    newX = np.zeros(X.shape)
    for k in range(X.shape[1]):
        bins = np.histogram(X[:,k], bins=5)
        newX[:,k] = bins
    
    return newX


def normData(X):
    u = X
    u[u == 0] = np.nan
    newX = np.zeros(X.shape)
    meanX = X.mean(axis=0)
    stdX = np.std(X, axis=0)
    
    for k in range(X.shape[1]):
        currentMean = meanX[k]
        currentStd = stdX[k]
        threshold = 3*currentStd + currentMean
        for ii in range(X.shape[0]):
            if u[ii,k] > threshold:
                newX[ii,k] = currentMean
            else:
                newX[ii,k] = u[ii,k]
                
    return (newX - np.min(newX))/(np.max(newX) - np.min(newX))


def reconstructionAndError(mat, randomRemovals=4, rank=9, iterations=10):
    u = mat
    length = mat.shape[0]
    width = mat.shape[1]

    t = u
    t[np.isnan(t)] = 0
    
    estimator = NMF(n_components=rank, solver='mu')
    w = estimator.fit_transform(t)
    h = estimator.components_
    predicted = np.dot(w,h)
    
    removalVec = np.zeros((randomRemovals,2))
    t2 = t
    avgRMSE = np.zeros((length,iterations))
    w2 = estimator.fit_transform(t2)
    h2 = estimator.components_
    predicted2 = np.dot(w2,h2)
    
    for k in range(iterations):
        count = 1
        skip = False
        
        while count < (randomRemovals):
            randX = rand.randint(0, length-1)
            randY = rand.randint(0, width-1)
            if not(np.isnan(t2[randX,randY])):
                removalVec[count,:] = [randX, randY]
                t2[randX, randY] = 0
                skip = True
            if skip:
                count = count+1
                skip = False
        
        close = 0
        for ii in range(randomRemovals):
            close = close + np.linalg.norm(
                t[int(removalVec[ii,0]),int(removalVec[ii,1])] - predicted2[int(removalVec[ii,0]),int(removalVec[ii,1])])
        avgRMSE[k] = np.sqrt(close/randomRemovals)
    
    finalRMSE = avgRMSE.mean()
    return w, h, predicted, finalRMSE


def reconstructionAndErrorSVD(mat, randomRemovals=4, threshold=0.9, iterations=10):
    u = mat
    length = mat.shape[0]
    width = mat.shape[1]

    t = u
    t[np.isnan(t)] = 0
    
    # estimator = NMF(n_components=rank, solver='mu')
    # w = estimator.fit_transform(t)
    # h = estimator.components_
    # predicted = np.dot(w,h)
    predicted = np.transpose(recommend(t,threshold=threshold))
    predicted2 = predicted.copy()
    
    removalVec = np.zeros((randomRemovals,2))
    t2 = t
    avgRMSE = np.zeros((length,iterations))
    # w2 = estimator.fit_transform(t2)
    # h2 = estimator.components_
    # predicted2 = np.dot(w2,h2)
    #predicted2 = recommend(t2,threshold=threshold)
    
    for k in range(iterations):
        count = 0
        skip = False
        
        while count < (randomRemovals):
            randX = rand.randint(0, length-1)
            randY = rand.randint(0, width-1)
            if not(np.isnan(t2[randX,randY])):
                removalVec[count,:] = [randX, randY]
                t2[randX, randY] = 0
                skip = True
            if skip:
                count = count+1
                skip = False
        
        close = 0
        for ii in range(randomRemovals):
            a = t[int(removalVec[ii,0]),int(removalVec[ii,1])]
            b = predicted2[int(removalVec[ii,0]),int(removalVec[ii,1])]
            close = close + np.linalg.norm(a - b)
        avgRMSE[k] = np.sqrt(close/randomRemovals)
    
    finalRMSE = avgRMSE.mean()
    return predicted, finalRMSE

In [11]:
# This cell contains two bare-bones classes that implement the two versions of our recommendation
# system (the NNMF version and the PCA by SVD version)
# The classes have all of the functions needed to test our implementations using GridSearchCV.
# An important note: GridSearchCV assumes that the highest score is the best. We're using the 
# RMSE between the original data and the reconstructed data, which means our "best" parameters
# will be the lowest. For this reason, the RMSE values are negated before they're returned.
# (the score() function returns -self.finalRMSE, not +self.finalRMSE)

class Recommend:
    def __init__(self, n_components=10, randomRemovals=4, iterations=10):
        self.rank = n_components
        self.randomRemovals = randomRemovals
        self.iterations = iterations
        return
    
    def fit(self,X,Y):
        w, h, predicted, finalRMSE = reconstructionAndError(X, randomRemovals=self.randomRemovals,iterations=self.iterations )
        self.finalRMSE = finalRMSE
        self.W = w
        self.h = h
        self.predicted = predicted
        return
    
    def transform(self,X):
        w, h, predicted, finalRMSE = reconstructionAndError(X, randomRemovals=self.randomRemovals,iterations=self.iterations )
        self.finalRMSE = finalRMSE
        self.W = w
        self.h = h
        self.predicted = predicted
        return
    
    def set_params(self,n_components=10, randomRemovals=4, iterations=10):
        self.rank = n_components
        self.randomRemovals = randomRemovals
        self.iterations = iterations
        # keys = params.keys()
        # if 'rank' in keys:
        #     Recommend.rank = params['rank']
        # if 'n_components' in keys:
        #     Recommend.rank = params['n_components']
        # if 'randomRemovals' in keys:
        #     Recommend.randomRemovals = params['randomRemovals']
        # if 'iterations' in keys:
        #     Recommend.iterations = params['iterations']
    
    def score(self, X, Y):
        return -self.finalRMSE

class RecommendSVD:
    def __init__(self, n_components=10, randomRemovals=4, iterations=10, threshold=0.9):
        self.rank = n_components
        self.randomRemovals = randomRemovals
        self.iterations = iterations
        self.threshold = threshold
        return
    
    def fit(self,X,Y):
        predicted, finalRMSE = reconstructionAndErrorSVD(
            X, randomRemovals=self.randomRemovals,iterations=self.iterations, threshold=self.threshold )
        self.finalRMSE = finalRMSE
        self.predicted = predicted
        return
    
    def transform(self,X):
        predicted, finalRMSE = reconstructionAndErrorSVD(
            X, randomRemovals=self.randomRemovals,iterations=self.iterations, threshold=self.threshold )
        self.finalRMSE = finalRMSE
        self.predicted = predicted
        return
    
    def set_params(self,n_components=10, randomRemovals=4, iterations=10, threshold=0.9):
        self.rank = n_components
        self.randomRemovals = randomRemovals
        self.iterations = iterations
        self.threshold=threshold
    
    def score(self, X, Y):
        return -self.finalRMSE

In [12]:
# Collect all the matlab-processed data, then split the data into train and test sets
dataSets = {}
trainTestSets = {}
for k in [5, 10, 15, 20]:
    # Construct file name (without extension)
    currentFile = 'UserArtists_' + str(k)
    
    # Load and store the data (there is only one variable in the file, ua_adj_transpose)
    currentSet = scipy.io.loadmat(currentFile+'.mat')['ua_adj_transpose']
    dataSets[currentFile] = currentSet
    
    # Split into train and test sets
    xtrain, xtest = train_test_split(currentSet, test_size=0.2)
    currentSplit = {'train': xtrain, 'test': xtest}
    
    # trainTestSets is a dictionary of dictionaries. Each dictionary it holds corresponds to
    # a train-test split of one of our four test datasets.
    trainTestSets[currentFile] = currentSplit


In [13]:
# Example of what dataSets holds (a 100xN array).
# The train/test splits are the same (a 100x0.8N matrix for training, 100x0.2N for testing)
dataSets['UserArtists_5'].shape

(100, 1346)

In [14]:
# This is where the parameterization goes. To test each method, just comment/uncomment the associated
# block. (This is being submitted with RecommendSVD() commented out, and Recommend() active)

# Pipelines are mutable (or at least functionally mutable when used with GridsearchCV), but they 
# need to be initialized to be used in a parameter grid search.
# The initialization is a set of ('name', Operation()) pairs. The 'name's are preserved 
# throughout the grid search, but the Operation()s are swapped out in an exaustive combination.
# Our pipeline only reduces the data (either by NNMF or PCA by SVD), so we include a field 
# called 'reducer'

pipeline = Pipeline((
    ('reducer', PCA()),
))

N_FEATURES_OPTIONS = [20, 40, 60, 80, 100]
THRESHOLDS = [0.1, 0.4, 0.7, 0.8, 0.9, 0.95, 0.99]
param_grid = [
    # {
    #     'reducer': [RecommendSVD()],
    #     'reducer__threshold': THRESHOLDS,
    # },
    {
        'reducer': [Recommend()],
        'reducer__n_components': N_FEATURES_OPTIONS,
    },
]
grid = GridSearchCV(pipeline, param_grid, n_jobs=-2)

In [15]:
# Get cv results for each dataset

# Holds all of the results and information for each dataset (can be slightly difficult to navigate)
gridResults = {}

# Holds the best parameter combinations for each dataset
# (the parameters for which results are reported)
bestCombinations = {}

# Holds the training cross-validation scores using the best parameters
# (conducted separately from the cvs in gridResults)
trainScores = {}

# Holds the testing cross-validation scores using the best parameters
testScores = {}
for key in trainTestSets.keys():
    xtrain = trainTestSets[key]['train']
    xtest = trainTestSets[key]['test']
    
    # This is where the exhaustive search is conducted. After .fit() returns, the 
    # results are in cv_results_, which holds 5-fold cv-scores for each combination
    # of parameters. The parameters that lead to the lowest RMSE are considdered
    # the "best" parameters, however, just because we have a low RMSE does not mean
    # our machine makes good music recommendations (that would require user testing)
    grid.fit(xtrain)
    gridResults[key] = pd.DataFrame(grid.cv_results_)
    bestCombinations[key] = grid.best_estimator_
    trainScores[key] = cross_val_score(grid.best_estimator_, xtrain, cv=5)
    testScores[key] = cross_val_score(grid.best_estimator_, xtest, cv=5)
    
testScores = pd.DataFrame(testScores)
trainScores = pd.DataFrame(trainScores)

In [16]:
# Display relevant CV scores (values are negative for GridSearchCV, interpret them as positive)
for key in testScores.keys():
    print(key)
    #print("Best FoV Threshold: " + str(bestCombinations[key]['reducer'].threshold))
    print("Best rank: " + str(bestCombinations[key]['reducer'].rank))
    print("Test data RMSE")
    print(testScores[key])
    print("Mean RMSE: " + str(testScores[key].mean()))
    print("Std RMSE: " + str(np.std(testScores[key])))
    print("")

UserArtists_5
Best rank: 0.4
Test data RMSE
0   -16.889572
1    -4.285606
2    -9.440983
3   -11.177004
4    -9.745557
Name: UserArtists_5, dtype: float64
Mean RMSE: -10.307744530776986
Std RMSE: 4.035084744190852

UserArtists_10
Best rank: 0.99
Test data RMSE
0    -6.693916
1    -4.307130
2    -4.359845
3    -2.999081
4   -11.764841
Name: UserArtists_10, dtype: float64
Mean RMSE: -6.02496244340764
Std RMSE: 3.1071925970644347

UserArtists_15
Best rank: 0.95
Test data RMSE
0   -4.663890
1   -2.196229
2   -2.916414
3   -2.242560
4   -8.150470
Name: UserArtists_15, dtype: float64
Mean RMSE: -4.03391260066157
Std RMSE: 2.244003038903447

UserArtists_20
Best rank: 0.95
Test data RMSE
0    -6.910756
1    -4.740739
2    -7.372109
3   -10.940431
4    -7.821062
Name: UserArtists_20, dtype: float64
Mean RMSE: -7.5570192606693185
Std RMSE: 1.9950176313280674

